# Network Analysis Dashboard

> In this notebook, we will create our graph called `G` that has `Author` nodes and numbered edges that represent a shared publication.

Do you remember how we tell nbdev what module to export to? Go ahead and add the default export directive to the cell below and tell it to export to a module called `graph`. When you're done, you can delete this markdown cell.

In [1]:
#| default_exp graph

Okay, the first thing we wanted to know was what kind of objects NetworkX expects. I decided to use the [tutorial provided on the documentation website](https://networkx.org/documentation/stable/tutorial.html) to get this information.

> " In NetworkX, nodes can be any hashable object e.g., a text string, an image, an XML object, another Graph, a customized node object, etc."

That's great because that means we can totally just use the Author object you made in pre-processing. Below I copied and pasted the author object you defined in that repo.

## One change

The only thing I changed here is that I added `publications` to the parameters. The default is an empty list.

In [2]:
#| export 
class Author:
    def __init__(self, first, middle, last, email=None, publications=[]):
        self.first = first
        self.middle = middle
        self.last = last
        self.email = email
        self.publications = publications

In [3]:
author = Author('Nicole', 'F', 'Brewer')
author

That display is pretty ugly. We want it to be more informative. Fortunately, ipython gives us a nice way to do this. 

## `__repr__`

If we add a function called `__repr__` to the author class, ipython will use it when we display our object like we did above.

In [24]:
#| export 
import pprint

Okay, let's write out this function first before making it part of the class.

In [25]:
def __repr__(self):
    return pprint.pformat(vars(self))

In [26]:
pprint.pformat(vars(author))

"{'email': None,\n 'first': 'Nicole',\n 'last': 'Brewer',\n 'middle': 'F',\n 'publications': []}"

In [27]:
print(__repr__(author))

{'email': None,
 'first': 'Nicole',
 'last': 'Brewer',
 'middle': 'F',
 'publications': []}


That looks good! We won't use this for our app, but it's handy to see when we're developing.

## patch

In order to add this function to the author object without having to return to a prior cell, we need to use a handy fastai utility. Let's see how that works. We will need to import the `@patch` decorator which tells tells the notebook processor that the function we write is intended to extend a class.

In [29]:
#| export
from fastcore.basics import patch

The function will be the same as before, except we will use `self:Class` instead of `self` in the function definition to tell the patch which class we are extending. Below is a modified example from the nbdev documentation.

In [14]:
#| export
class Number:
    "A number."
    def __init__(self, num): self.num = num

In [15]:
number = Number(5)
number

In [18]:
@patch
def __repr__(self:Number):
    return f'Number({self.num})'

In [19]:
number = Number(5)
number

Number(5)

In [11]:
class Number:
    "A number."
    def __init__(self, num):
        self.num = num
    def __repr__(self):
        return f'Number({self.num})'

In [12]:
number = Number(5)
number

Number(5)

That looks a lot better right? Now you can use this example about Number objects to modify the cell below so it actually gets attached to our `Author` object. 

In [20]:
#| export
# two things need to be added to create a patch for Author
@patch
def __repr__(self:Author):
    return pprint.pformat(vars(self))

Great job! We should start by creating a few `Author` objects. In the process we can verify that our patch worked.

In [21]:
a1 = Author('A', 'B', 'Carlson', publications = [1,2,3])
a2 = Author('B', 'C', 'Dawson', publications = [3, 4, 5 ,6])
a3 = Author('C', 'D', 'Elfson', publications = [1, 5, 7, 8])
a4 = Author('D', 'E', 'Fitzgerald', publications = [1, 5, 9, 10])
a5 = Author('E', 'F', 'Gerard', publications = [4, 11, 12])
a5

{'email': None,
 'first': 'E',
 'last': 'Gerard',
 'middle': 'F',
 'publications': [4, 11, 12]}

In [79]:
authors = [a1, a2, a3, a4, a5]

Now let's use NetworkX to create a graph `G`.

In [44]:
import networkx as nx

G = nx.Graph()

Now let's add our `Author` objects as nodes on the graph. We can add nodes individually.

In [45]:
G.add_node(a1)

Or with any iterable (like a list) with the function `add_nodes_from`.

In [46]:
G.add_nodes_from(authors)

**Note**: if you add the same node or edge more than once, NetworkX quietly ignores any that are already present. That's good for us.

Let's see what we have so far.

In [47]:
G.nodes

NodeView(({'email': None,
 'first': 'A',
 'last': 'Carlson',
 'middle': 'B',
 'publications': [1, 2, 3]}, {'email': None,
 'first': 'B',
 'last': 'Dawson',
 'middle': 'C',
 'publications': [3, 4, 5, 6]}, {'email': None,
 'first': 'C',
 'last': 'Elfson',
 'middle': 'D',
 'publications': [1, 5, 7, 8]}, {'email': None,
 'first': 'D',
 'last': 'Fitzgerald',
 'middle': 'E',
 'publications': [1, 5, 9, 10]}, {'email': None,
 'first': 'E',
 'last': 'Gerard',
 'middle': 'F',
 'publications': [4, 11, 12]}))

Okay let's add an edge. First we will do this manually and then we will try with a loop. To do this manually, can you tell me two Authors that share a publication? When two authors share a publication, we want to draw and edge between them.

In [48]:
G.add_edge(a3, a4) # add two authors that share a publication

Okay. Now we need to draw the edgets between these author objects! In our case, we want the edges to be added every time two authors share a publication. Right now (because we only have `Author` objects and not `Publication` objects) we will just have to loop through the authors and their publications to check for a match. 

> Note: if we did have `Publication` objects, we could loop through the publication objects on the outside and the authors on the inside of the loop. And we could actually set those `Publication` objects as our edges using the notation `G.add_edge(a1, a2, object=<shared_publication>)`.

What do we need to iterate through on the outside of the loop? What about the inside?

You may need to use the [range](https://www.w3schools.com/python/ref_func_range.asp)  function.

In [99]:
############### 1ST ITERATION #######################
# write a loop that loops through authors and publications
# to add edges to our graph

#####################
"""for i in range(len(authors)):
    authors = authors[i]
    
    for j in range(len(number)):
        publication = publication[j]

        if same(author, publication):
            G.add_edge(author, publication)"""
    

'for i in range(len(authors)):\n    authors = authors[i]\n    \n    for j in range(len(number)):\n        publication = publication[j]\n\n        if same(author, publication):\n            G.add_edge(author, publication)'

In [95]:
for publication in a1.publications:
    
    for publication_a2 in a2.publications:
        print(f"checking if {publication} and {publication_a2} match")
        if publication == publication_a2:
            print("match")
           

checking if 1 and 3 match
checking if 1 and 4 match
checking if 1 and 5 match
checking if 1 and 6 match
checking if 2 and 3 match
checking if 2 and 4 match
checking if 2 and 5 match
checking if 2 and 6 match
checking if 3 and 3 match
match
checking if 3 and 4 match
checking if 3 and 5 match
checking if 3 and 6 match


In [87]:
for author in authors:
    for publication in author.publications:
        print (publication)

1
2
3
3
4
5
6
1
5
7
8
1
5
9
10
4
11
12


In [88]:
authors[index]

{'email': None,
 'first': 'E',
 'last': 'Gerard',
 'middle': 'F',
 'publications': [4, 11, 12]}

In [96]:
"""for index_a in range(len(authors)):
    
        for publication in authors[index].publications:
            print (publication)
            print(f"Comparing a{index_a} with a{index_b}")
        """

'for index_a in range(len(authors)):\n    \n        for publication in authors[index].publications:\n            print (publication)\n            print(f"Comparing a{index_a} with a{index_b}")\n        '

In [92]:
#| export

for index_a in range(len(authors)):
    for index_b in range(index_a + 1, len(authors)):
        author_a = authors[index_a]
        author_b = authors[index_b]
        
        for publication_a in author_a.publications:
            for publication_b in author_b.publications:
                if publication_a == publication_b:
                    G.add_edge(author, publication)
                    print(f"Match found between a{index_a+1} and a{index_b+1} for publication {publication_a}")

Match found between a1 and a2 for publication 3
Match found between a1 and a3 for publication 1
Match found between a1 and a4 for publication 1
Match found between a2 and a3 for publication 5
Match found between a2 and a4 for publication 5
Match found between a2 and a5 for publication 4
Match found between a3 and a4 for publication 1
Match found between a3 and a4 for publication 5


In [93]:
G.edges

EdgeView([({'email': None,
 'first': 'C',
 'last': 'Elfson',
 'middle': 'D',
 'publications': [1, 5, 7, 8]}, {'email': None,
 'first': 'D',
 'last': 'Fitzgerald',
 'middle': 'E',
 'publications': [1, 5, 9, 10]}), ({'email': None,
 'first': 'E',
 'last': 'Gerard',
 'middle': 'F',
 'publications': [4, 11, 12]}, 4)])

## Plotting

You probably want to do more advanced plotting in a plotting notebook, but we can at least view what we have so far for fun!

In [113]:
from ipycytoscape import CytoscapeWidget

cyto = CytoscapeWidget()
cyto.graph.add_graph_from_networkx(G)
cyto

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [100]:
class Color:
    def __init__(self, color): self.color = color
    def _repr_markdown_(self):
        style = f'background-color: {self.color}; width: 50px; height: 50px; margin: 10px'
        return f'<div style="{style}"></div>'

In [ ]:
### G.remove_node(2)

## Export to `dashboard` pkg

Let's use the nbdev_export function to export this notebook to a package called `dashboard`. You will need to add parameters to the function below.

In [107]:
from nbdev.export import nb_export

nb_export('create_graph.ipynb', 'dashboard')